In [3]:
!python -m ipykernel install --name=MyPYL

Installed kernelspec MyPYL in C:\ProgramData\jupyter\kernels\mypyl


In [1]:
from segment.modules.semibase import Base
from segment.modules.semseg.deeplabv3plus import DeepLabV3Plus
import torch
from torchvision import transforms as T
from torchvision.transforms import functional as F
import torch.nn as nn
from PIL import Image
import numpy as np
from segment.util import  color_map
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

D:\Dev_soft\Anaconda3\envs\MyPYL\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
D:\Dev_soft\Anaconda3\envs\MyPYL\lib\site-packages\pytorch_lightning\utilities\distributed.py:258: LightningDeprecationWarning: `pytorch_lightning.utilities.distributed.rank_zero_only` has been deprecated in v1.8.1 and will be removed in v2.0.0. You can import it from `pytorch_lightning.utilities` instead.
  rank_zero_deprecation(


In [2]:
num_classes = 3
ckpt_path = '../temp/val_OD_dice=0.847466.ckpt'

cmap = color_map('eye')

model = DeepLabV3Plus('resnet50',num_classes)

sd = torch.load(ckpt_path, map_location='cpu')['state_dict']
if 'state_dict' in sd:
    # If 'state_dict' exists, use it directly
    sd = sd['state_dict']
new_state_dict = {}
for key, value in sd.items():
    # if not key.startswith('module.'):  # 如果关键字没有"module."前缀，加上该前缀
    #     if 'module.' + key in model.state_dict():
    #         # 模型在多GPU上训练并保存，加载权重时加上"module."前缀
    #         key = 'module.' + key
    key = key.replace('model.', '')
    new_state_dict[key] = value
model.load_state_dict(new_state_dict)
model.eval()


image_path = './T0001.jpg'
mask_path = './T0001.bmp'
image_tensor = T.ToTensor()(Image.open(image_path)).unsqueeze(dim=0)
mask_tensor = T.ToTensor()(Image.open(mask_path))


output = model(image_tensor)
backbone_feat, logits = output['backbone_features'], output['out']

In [25]:
# Take the first sample in batch
# Take the first sample in batch
logits_sample = logits[0].detach().numpy()
# 将logits的形状转换为(512*512, 3)
logits_reshaped = logits_sample.reshape(-1, 3)
# print(logits_reshaped.shape)
# 使用t-SNE进行降维，降维后的结果为(512*512, 2)
tsne = TSNE(n_components=2)
logits_tsne = tsne.fit_transform(logits_reshaped)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Windows\Temp\ipykernel_97084\961911661.py:9 in <module>                                       │
│                                                                                                  │
│ [Errno 2] No such file or directory: 'C:\\Windows\\Temp\\ipykernel_97084\\961911661.py'          │
│                                                                                                  │
│ C:\Users\杨钊灿\AppData\Roaming\Python\Python39\site-packages\sklearn\manifold\_t_sne.py:1119 in │
│ fit_transform                                                                                    │
│                                                                                                  │
│   1116 │   │   """                                                                               │
│   1117 │   │   self._validate_params()                                                           │
│   1118 │   │   self._check_params_vs_input(X)                                                    │
│ ❱ 1119 │   │   embedding = self._fit(X)                                                          │
│   1120 │   │   self.embedding_ = embedding                                                       │
│   1121 │   │   return self.embedding_                                                            │
│   1122                                                                                           │
│                                                                                                  │
│ C:\Users\杨钊灿\AppData\Roaming\Python\Python39\site-packages\sklearn\manifold\_t_sne.py:995 in  │
│ _fit                                                                                             │
│                                                                                                  │
│    992 │   │   │   )                                                                             │
│    993 │   │   │   # Always output a numpy array, no matter what is configured globally          │
│    994 │   │   │   pca.set_output(transform="default")                                           │
│ ❱  995 │   │   │   X_embedded = pca.fit_transform(X).astype(np.float32, copy=False)              │
│    996 │   │   │   # PCA is rescaled so that PC1 has standard deviation 1e-4 which is            │
│    997 │   │   │   # the default value for random initialization. See issue #18018.              │
│    998 │   │   │   X_embedded = X_embedded / np.std(X_embedded[:, 0]) * 1e-4                     │
│                                                                                                  │
│ C:\Users\杨钊灿\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\_set_output.py:142   │
│ in wrapped                                                                                       │
│                                                                                                  │
│   139 │                                                                                          │
│   140 │   @wraps(f)                                                                              │
│   141 │   def wrapped(self, X, *args, **kwargs):                                                 │
│ ❱ 142 │   │   data_to_wrap = f(self, X, *args, **kwargs)                                         │
│   143 │   │   if isinstance(data_to_wrap, tuple):                                                │
│   144 │   │   │   # only wrap the first output for cross decomposition                           │
│   145 │   │   │   return (                                                                       │
│                                                                                                  │
│ C:\Users\杨钊灿\AppData\Roaming\Python\Python39\site-packages\sklearn\decomposition\_pca.py:462  │
│ in fit_transform                                                       

In [23]:
# 获取每个点对应的类别
# 这里假设logits的形状中3代表三个类别，你可以根据实际情况修改这里的代码
# labels = np.argmax(logits_reshaped, axis=1)
#
# # 绘制降维后的结果
# plt.scatter(logits_tsne[:, 0], logits_tsne[:, 1], c=labels, cmap='viridis')
# plt.colorbar()
# plt.show()
